## Ipsen perturbation bound - inducing sparsity in the original matrix

In the Ipsen SDP relaxation derivation, we start with the matrix to be perturbed (rank-one given by its factor rank_one_vector), then multiply it on the left and right by a diagonal matrix (inducing sparsity). This is relaxed to a Hadamard product with a PSD matrix S (sparsity-inducing), whose elements are in [0, 1].

We are sparsifying and reducing the magnitude of rank_one_vector, so we need to unit norm again. We would like S to be multiplied by some unknown scalar c, so that the L2 norm in the denominator goes to one. This is still the same problem after squaring the expression, then we square root the relation that we would want to have, and we see that we can use the power cone P_3^(1/2) (that is, if the inequality is effective at bounding ||S||).

In [1]:
using COSMO, LinearAlgebra, Random, CSV, Tables, Plots

In [2]:
data = CSV.File("../NC-Data.csv") |> Tables.matrix;
raw_data = sqrt(data)';
n = size(data, 1);
N = n * n;
# All close.
maximum(abs.(raw_data' * raw_data .- data))

4.884981308350689e-15

In [3]:
selected_data = [3; 83; 85];
opt_inds = [79; 80; 81; 84];
k = 7

7

In [4]:
U = svd(raw_data[:, selected_data]).U[:, 1];
rank_one_vector = raw_data' * U;

In [5]:
residual_data = raw_data .- (U * U' * raw_data);

In [6]:
mapslices(norm, raw_data, dims=1)

1×101 Array{Float64,2}:
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  …  1.0  1.0  1.0  1.0  1.0  1.0  1.0

In [7]:
mapslices(norm, residual_data, dims=1)

1×101 Array{Float64,2}:
 0.998223  0.986038  0.846852  0.949473  …  0.958106  0.996008  0.92525

In [24]:
size([vec(rank_one_vector * rank_one_vector')';
      zeros(1, N);
      vec(Matrix(1.0I, n, n))'])

(3, 10201)

In [25]:
[zeros(3, 10201) zeros(3, aux)]

3×10303 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [75]:
model = COSMO.Model()

# Number of auxiliary helper variables
aux = n+1
psd_c = COSMO.Constraint([Matrix(1.0I, N, N) zeros(N, aux)], zeros(N), COSMO.PsdCone)
nn_c = COSMO.Constraint([Matrix(1.0I, N, N) zeros(N, aux)], zeros(N), COSMO.Nonnegatives)
power_c = COSMO.Constraint(
    # Three arguments:
    # x_1 = Tr Svv^T = sum_i sum_j S_{ij} v_i v_j
    # x_2 = k^2
    # x_3 = Tr S (vectorize the identity matrix)
    # Constraint: x1 * x2 >= x3^2
    hcat([vec(rank_one_vector * rank_one_vector')';
            zeros(1, N);
            vec(Matrix(1.0I, n, n))'],
         zeros(3, aux)),
    [0.; k^2; 0],
    COSMO.PowerCone(0.5))
# One row: The diag entry may be zero or nonzero.
row_c = repeat(Matrix(-1.0I, n, n), inner=(1, n))
for i = 1:n
    row_c[i, 1 + (n+1)*(i-1)] = k-1
end
row_c = COSMO.Constraint([row_c zeros(n, aux)], zeros(n), COSMO.ZeroSet(n))

# Power cone: To scale the diagonals to unit box, we should take the sqrt/harmonic mean.
harmonic_c = []
for i = 1:n
    x1 = zeros(1, N+aux)
    x1[N + i] = 1.
    x2 = zeros(1, N+aux)
    x2[end] = 1.
    x3 = zeros(1, N+aux)
    x3[1 + (n+1)*(i-1)] = 1.
    push!(harmonic_c, COSMO.Constraint(
        [x1; x2; x3],
        zeros(3),
        COSMO.PowerCone(0.5)))
end
# Aux variables are contained in the unit box. Small-n constraints.
box_c = COSMO.Constraint(
    [zeros(n, N) Matrix(1.0I, n, n) zeros(n, 1)],
    zeros(n),
    COSMO.Box(zeros(n), ones(n)))

rank_one_variance = rank_one_vector .^ 2

assemble!(
    model,
    zeros(N+aux, N+aux),
    [-vec((residual_data' * residual_data) .* (rank_one_vector * rank_one_vector')); -rank_one_variance; 0],
    Vector{COSMO.Constraint{Float64}}(vcat([psd_c, nn_c, power_c, row_c, box_c], harmonic_c)),
    settings = COSMO.Settings(time_limit = 60))

In [76]:
result = COSMO.optimize!(model)

>>> COSMO - Results
Status: 

Max_iter_reached
Iterations: 5000 (incl. 0 safeguarding iterations)
Optimal Objective: -248.7
Runtime: 25793.34ms
Setup Time: 21.33ms

Avg Iter Time: 5.15ms

In [77]:
maximum(abs.(Diagonal(reshape(result.x[1:N], n, n))))

34.23481095623675

In [78]:
# Is the bound tight? Unit-norming of the vector in matrix-vector multiplication.
# We need more bounds which bring this closer to 1.
mat = reshape(result.x[1:N], n, n)
sum(Diagonal(mat)) ^ 2 / sum(Diagonal(mat) .* (rank_one_vector .^ 2))

352.6323961866911

In [79]:
maximum(Diagonal(mat))

34.23481095623675

In [80]:
minimum(Diagonal(mat))

-0.02730715791107616

In [81]:
sum(min.(Diagonal(mat), 0.))

-2.0616702855364446

In [82]:
result.x

10303-element Array{Float64,1}:
   -0.026822949943890888
   -0.0006575253644767372
    0.0013928698980422444
   -0.0013717854613016912
   -0.0021952180034445893
   -0.00035470860912303583
    0.001768639458249956
    0.000691304207184118
    0.000728233263617193
    0.0005720995127573214
    ⋮
    0.9998397902564627
    1.000109816054214
    0.9999959778464803
    0.999710834479926
    1.0000200361974105
    1.000018684535188
    1.0001235042086072
    1.0000257072747845
 3133.1107845776696

In [84]:
result.x[N + 1]

1.0002749568428562